# Server

In [12]:
import requests.exceptions
import tekore as tk
from dotenv import load_dotenv
import httpx

load_dotenv()

conf = tk.config_from_environment()
cred = tk.Credentials(*conf)

# trans = httpx.HTTPTransport(retries=3)
# client = httpx.Client(timeout=30, transport=trans)
# sender = tk.SyncSender(client=client)
spotify = tk.Spotify()

host = '127.0.0.1'
port = 5000

In [13]:
from os import environ as env
import json
from authentication.spotify_server import SpotifyServer
try:
    with open(".spotify_credentials.json", "r") as infile:
        token_dict = json.load(infile)
        token_dict["scope"] = " ".join(token_dict["scope"])
        spotify.token = tk.Token(token_dict, token_dict["uses_pkce"])
        spotify.token = cred.refresh(spotify.token)
except Exception as e:
    print(e)
    app = SpotifyServer(host, port, spotify, cred)
    spotify.token = app.spawn_single_use_server()

In [14]:
from typing import List
token: tk.Token = spotify.token
scopes: List[str] = list(token.scope)
token_dict = {
    "token_type": token.token_type,
    "access_token": token.access_token,
    "refresh_token": token.refresh_token,
    "scope": scopes[0].replace("[", "").replace("]", "").replace("'", "").split(", ") if len(scopes) == 0 else scopes,
    "expires_at": token.expires_at,
    "uses_pkce": token.uses_pkce,
    "expires_in": 0
}
json_object = json.dumps(token_dict, indent=4)
with open(".spotify_credentials.json", "w") as outfile:
    outfile.write(json_object)

# Spotify API

In [15]:
spotify.token = cred.refresh(spotify.token)

In [16]:
from project.random_track import RandomTrack

rndTrack = RandomTrack(spotify, cred)
tracks = rndTrack.random_tracks()
[track.name for track in tracks]

['God Was Never on Your Side',
 'Going Hunting',
 'The Game (Triple H)',
 'Ghost From The Barrow',
 "God's Wrath",
 'Götterdämmerung',
 'Get It Crackin - Sefa Remix',
 'VICTORY OR DEATH (WE GAVE ‘EM HELL) (feat. Campino of Die Toten Hosen)',
 'Let It Go',
 'goosebumps',
 'Schwarz Tot Gold',
 'Let It Go',
 'I Gotta Feeling',
 'Gegen den Wind - Piano Version',
 'Das ist alles von der Kunstfreiheit gedeckt',
 "You're Gonna Go Far, Kid",
 "Gangsta's Paradise",
 "I'm Good (Blue)",
 'My Own Grave',
 'Gimme! Gimme! Gimme! (A Man After Midnight)',
 'Godzilla (feat. Juice WRLD)',
 'good 4 u',
 'GigaChad Theme - Phonk House Version',
 'Bloody Mary',
 'Green Green Grass',
 'Get Shaky',
 'Give It To Me',
 'Grenade',
 'Glimpse of Us',
 'Glatteis',
 'Gold',
 "Thank God It's Christmas - Non-Album Single",
 "God's Plan",
 'Go Solo',
 'Gad',
 "I Ain't Worried",
 'Give Me Everything (feat. Ne-Yo, Afrojack & Nayer)',
 'Give It To Me - Full Vocal Mix',
 '61 Grad (Deep House RMX)',
 'Geschenk',
 'How Do I 

In [17]:
from project.util import Partition

artist_ids = [artist.id for track in tracks for artist in track.artists]
artist_id_partitions = Partition(artist_ids)
artists = [spotify.artists(ids) for ids in artist_id_partitions]
artists: List[tk.model.FullArtist] = list(artist for partition in artists for artist in partition)
len(artists)

68

In [18]:
import pylast
from authentication.lastfm_credentials import LastFmCredentials
from lastfm import LastFmScraper, LastFmProxy

creds = LastFmCredentials(env["LASTFM_API_KEY"], env["LASTFM_SHARED_SECRET"])
last_network = pylast.LastFMNetwork(
    api_key=creds.api_key,
    api_secret=creds.shared_secret,
)

lastfm = LastFmProxy(last_network, LastFmScraper())

In [19]:
import track as model
analyzed_tracks = model.AnalyzedTracks(tracks, artists)

In [20]:
from project.track import TaggedTrack
for track in analyzed_tracks.tracks:
    try:
        tags = TaggedTrack(lastfm, track.name, track.artist_names).tags()
        if len(tags) == 0:
            raise Exception("No tags")
    except Exception as e:
        print(e, track.id, track.name, track.artist_names)
        continue
    track.tags = tags

No tags 5ZNa6S7agmCmaEnxIHE73m Get Shaky ['TurboKevin', 'KXXMA']
No tags 4D7qV8IeK3Rt5zhqRTZHSN Geschenk ['Chiko69']


In [22]:
from time import sleep

track_ids = [track.id for track in analyzed_tracks.tracks]
track_id_partitions = Partition(track_ids)
features = [spotify.tracks_audio_features(ids) for ids in track_id_partitions]
features = [feature for partition in features for feature in partition]

for track in analyzed_tracks.tracks:
    successful = False
    analysis = None
    while not successful:
        try:
            analysis = spotify.track_audio_analysis(track_id=track.id)
            successful = True
        except httpx.HTTPError:
            print('ReadTimeout')
            successful = False
            sleep(3)

    feature = [feature for feature in features if feature.id == track.id][0]
    track.acousticness = feature.acousticness
    track.pitches = { str(segment.start):[p for p in segment.pitches] for segment in analysis.segments}
    track.loudness = feature.loudness
    track.energy = feature.energy
    track.danceability = feature.danceability
    track.mode = feature.mode
    track.instrumentalness = feature.instrumentalness
    track.key = feature.key
    track.liveness = feature.liveness
    track.tempo = feature.tempo
    track.time_signature = feature.time_signature
    track.valence = feature.valence

ReadTimeout
ReadTimeout
ReadTimeout
ReadTimeout
ReadTimeout
ReadTimeout
ReadTimeout


# Firebase

In [24]:
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.ApplicationDefault()
firebase_admin.initialize_app()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [26]:
from google.cloud.firestore_v1 import Client

firestore_client: Client = firestore.client()
analyzed_tracks.upsert(firestore_client.collection("tracks"))

In [25]:
analyzed_tracks.tracks[0]

AnalyzedTrack(id=3zeQSYzaN9kLVypKWr6yUi, name=God Was Never on Your Side, duration=261186, artist_genres={'album rock', 'speed metal', 'metal', 'hard rock', 'rock'})